# rdag_mle_figures
### code to make the figures in
### "Symmetries in directed Gaussian graphical models"
### by Visu Makam, Philipp Reichenbach, and Anna Seigal 

In [1]:
import numpy as np
import seaborn as sns
import networkx as nx
import random
import matplotlib.pyplot as plt
import pandas as pd
import time

In [2]:
# generate aDAG on "n_vertices" many nodes, edges present with probability "density"
# graph colouring with n_colours many edge colours and max no. vertex colours subject to compatibility
# edge weights uniform on [-1 -0.25] \cup [0.25 1], node weights uniform on [0,1]
# sample "n_samples" many times from the RDAG distribution
# compute both RDAG and uncoloured DAG MLE and compute l_2 distance to true parameters
def mle_compare(n_vertices, density, n_colours, n_samples):
    # build DAG 
    Gu = nx.erdos_renyi_graph(n_vertices,density)
    G = nx.DiGraph()
    G.add_nodes_from(Gu)
    G.add_edges_from(Gu.edges())
    G = G.reverse() # edges point from higher index to lower index 
    
    # assign colours to edges
    for (u, v) in G.edges():
        G.edges[u,v]['colour'] = random.randint(0,n_colours-1)
        
    # sample edge weights uniformly from [-1, -0.25] \cup [0.25, 1]
    us = np.random.uniform(-0.75,0.75,n_colours)
    us[us > 0] = us[us > 0] + 0.25
    us[us < 0] = us[us < 0] - 0.25
    weights = us
    
    # assign weights to edges according to colours
    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = weights[G.edges[u,v]['colour']]
        
    # list the parent relationship colours for each vertex
    prc = []
    for u in G.nodes():
        prcu = []
        for v in G.nodes():
            if (v,u) in G.edges():
                prcu.append(G.edges[v,u]["colour"])
        prc.append(prcu)
        
    # colour the vertices so that the colouring is compatible
    # assign weights to node colours 
    K = nx.complete_graph(n_vertices)
    H = nx.empty_graph(n_vertices)
    for (u,v) in K.edges():
        if bool(set(prc[u]) & set(prc[v])):
            H.add_edge(u,v)
    list_of_cc = sorted(nx.connected_components(H), key=len, reverse=True)
    node_weights = np.random.uniform(0,1,len(list_of_cc))
    for u in G.nodes():
        for c in range(len(list_of_cc)):
            if u in list_of_cc[c]:
                G.nodes[u]["colour"] = c
                G.nodes[u]["weight"] = node_weights[c]
                
    # generate lambda and omega matrices
    mat = nx.adjacency_matrix(G,range(n_vertices))
    lamb = mat.todense().T
    omega = np.zeros((n_vertices,n_vertices))
    for i in range(n_vertices):
        omega[i,i] = G.nodes[i]["weight"]
    ide = np.identity(n_vertices)
    idlam = np.linalg.inv(ide-lamb)
    cov1 = np.dot(idlam,omega)
    cov = np.dot(cov1, idlam.T)
    
    # generate sample data
    sample_data = np.zeros((n_vertices,n_samples))
    for i in range(n_samples):
        sample_data[:,i] = np.random.multivariate_normal(np.zeros(n_vertices),cov)
    sample_data.shape 
    
    # build augmented sample matrices, one for each vertex colour
    MY0 = [];
    MY = [];
    betas = [];
    for s in range(len(list_of_cc)):
        alpha = list_of_cc[s] # the vertices of colour s 
        alpha_vector = np.array(list(alpha))
        MYs0 = [];
        for u in alpha:
            MYs0 = np.concatenate((MYs0, sample_data[u,:]))
        MY0.append(MYs0)
        beta_multiset = [];
        for u in alpha:
            for v in G.nodes():
                if (v,u) in G.edges():
                    beta_multiset.append(G.edges[v,u]['colour'])
        beta = set(beta_multiset)
        betas.append(beta)
        beta_vector = np.array(list(beta))
        MYsblocks = [];
        for u in alpha:
            MYsblock = np.zeros((len(beta),n_samples))
            for v in G.nodes():
                if (v,u) in G.edges():
                    MYsblock[np.where(beta_vector == G.edges[v,u]['colour'])[0],:] += sample_data[v,:]
            MYsblocks.append(MYsblock)  
        MYs = np.zeros((len(beta),0));
        for i in range(len(alpha)):
            MYs = np.concatenate((MYs,MYsblocks[i]),axis=1)
        MY.append(MYs)
        
    # compute the RDAG MLE
    node_weights_hat = np.zeros(len(list_of_cc));
    weights_hat = np.zeros(n_colours);
    for s in range(len(list_of_cc)):
        beta = betas[s]
        beta_vector = np.array(list(beta))
        b = MY0[s];
        if MY[s].shape[0] > 0:
            A = MY[s].T;
            pinvA = np.linalg.pinv(A)
            weights_beta_s = np.dot(pinvA,b)
            weights_hat[beta_vector] = weights_beta_s
            diff = np.linalg.norm(np.dot(A,weights_beta_s)-b)**2
        else:
            diff = np.linalg.norm(b)**2
        alpha_s = len(list_of_cc[s])
        weights_alpha_s = (1/(alpha_s*n_samples))*diff
        node_weights_hat[s] = weights_alpha_s
        
    # compute uncoloured DAG MLE
    usual_DAG_node_weights = np.zeros(n_vertices)
    usual_DAG_weights = [] 
    for u in G.nodes():
        b = sample_data[u,:]
        AT = np.zeros((0,n_samples))
        for v in G.nodes():
                if (v,u) in G.edges():
                    AT = np.vstack((AT, sample_data[v,:]))
        if AT.shape[0] > 0:
            A = AT.T;
            pinvA = np.linalg.pinv(A)
            weights_beta_s = np.dot(pinvA,b)
            usual_DAG_weights.append(weights_beta_s)
            diff = np.linalg.norm(np.dot(A,weights_beta_s)-b)**2
        else:
            diff = np.linalg.norm(b)**2
        usual_DAG_node_weights[u] = (1/n_samples)*diff
    if usual_DAG_weights != []:
        usual_DAG_weights = np.concatenate(usual_DAG_weights)
    
    # duplicate RDAG weights to compare to uncoloured DAG output
    node_weights_comparable = np.zeros(n_vertices)
    node_weights_hat_comparable = np.zeros(n_vertices)
    for u in G.nodes():
        c = G.nodes[u]["colour"]
        node_weights_comparable[u] = node_weights[c]
        node_weights_hat_comparable[u] = node_weights_hat[c]
    weights_comparable = []
    weights_hat_comparable = []
    for u in G.nodes():
        weights_u = []
        weights_hat_u = []
        for v in G.nodes():
            if (v,u) in G.edges():
                c = G.edges[v,u]["colour"]
                weights_u.append(weights[c])
                weights_hat_u.append(weights_hat[c])
        weights_comparable.append(weights_u)
        weights_hat_comparable.append(weights_hat_u)
        
    # compute distance between MLE and true parameters
    weights_comparable = np.concatenate(weights_comparable)
    weights_hat_comparable = np.concatenate(weights_hat_comparable)
    edge_dist_RDAG = np.linalg.norm(weights_hat_comparable - weights_comparable)**2
    node_dist_RDAG = np.linalg.norm(node_weights_hat_comparable - node_weights_comparable)**2
    edge_dist_DAG = np.linalg.norm(usual_DAG_weights - weights_comparable)**2
    node_dist_DAG = np.linalg.norm(usual_DAG_node_weights - node_weights_comparable)**2
    mles = np.zeros(2)
    mles[0] = edge_dist_RDAG + node_dist_RDAG
    mles[1] = edge_dist_DAG + node_dist_DAG
    return mles

In [3]:
# time mle_compare
start = time.time()
mle_compare(10,0.5,5,100)
end = time.time()
end-start

0.025613069534301758

In [ ]:
# vary number of samples 
mles = pd.DataFrame(columns = ["vertices", "density", "no. colours", "samples", "colours", "log MLE diff"])
n_vertices = 10
density = 0.5
n_colours = 5
for n_samples in [10, 50, 100, 1000, 10000]:
    print(n_samples)
    for trials in range(50):
        a = mle_compare(n_vertices,density,n_colours,n_samples)
        v = [n_vertices,density,n_colours,n_samples,"Yes",np.log(a[0])]
        mles.loc[len(mles.index)] = v
        w = [n_vertices,density,n_colours,n_samples,"No",np.log(a[1])]
        mles.loc[len(mles.index)] = w

In [ ]:
# plot figure 1
ax = sns.violinplot(x="samples", y="log MLE diff", hue="colours", data=mles, palette="muted")
plt.savefig('violin1.png', dpi=300)

In [ ]:
# vary number of colours 
mles = pd.DataFrame(columns = ["vertices", "density", "no. colours", "samples", "log MLE diff"])
n_samples = 100
n_vertices = 10
density = 0.5
for n_colours in [2, 5, 10, 50, 100]:
    print(n_colours)
    for trials in range(50):
        a = mle_compare(n_vertices,density,n_colours,n_samples)
        v = [n_vertices,density,n_colours,n_samples,np.log(a[0])]
        mles.loc[len(mles.index)] = v
        w = [n_vertices,density,"none",n_samples,np.log(a[1])]
        mles.loc[len(mles.index)] = w

In [ ]:
# plot figure 2
my_order=[ 2, 5, 10, 50, 100, "none"]
b = sns.color_palette('muted')[0]
o = sns.color_palette('muted')[1]
pal = [b,b,b,b,b,o]
ax = sns.violinplot(x="no. colours", y="log MLE diff", data=mles, palette=pal,order=my_order)
plt.savefig('violin2.png', dpi=300)

In [ ]:
# vary edge density
mles = pd.DataFrame(columns = ["vertices", "density", "no. colours", "samples", "colours", "log MLE diff"])
n_vertices = 10
density = 0.5
n_colours = 5
n_samples = 100
for density in [0.1, 0.3, 0.5, 0.7, 0.9]:
    print(density)
    for trials in range(50):
        a = mle_compare(n_vertices,density,n_colours,n_samples)
        v = [n_vertices,density,n_colours,n_samples,"Yes",np.log(a[0])]
        mles.loc[len(mles.index)] = v
        w = [n_vertices,density,n_colours,n_samples,"No",np.log(a[1])]
        mles.loc[len(mles.index)] = w

In [ ]:
# plot figure 3
ax = sns.violinplot(x="density", y="log MLE diff", hue="colours", data=mles, palette="muted")
plt.savefig('violin3.png', dpi=300)